# 데코레이터

데코레이터는 사전적인 의미 그대로 해석하자면 함수를 장식한다는 개념입니다. 기존에 이미 작성된 코드에 새로운 기능을 추가하여 장식하는 개념으로 기존의 코드를 수정하지 않고 새로운 기능을 확장하거나 할때 사용되는 개념입니다.

사실 데코레이터를 얘기하자면 파이썬의 함수의 특성과 클로저에 대해서 이해해야 하는데 간단하게 예제를 보면서 알아봅시다.

### 파이썬 함수의 특성

In [1]:
def add(a, b):
    return a + b

r2 = add(1, 5) # 괄호를 붙여서 인자값을 넘김
r1 = add # 괄호를 붙이지 않고 함수명 자체를 넘김

r1(1, 5) # r1 은 함수처럼 사용할 수 있음 (C++ 함수포인터 처럼 동작)

6

### 클로저
* 파이썬에서 함수는 일급함수라고 이전 강좌에서 얘기했습니다.
* 이때 일급함수에서의 네임바인딩 기술을 클로저라고 하는데 쉽게 지역변수와 프로그램 코드를 묶어서 관리하며 사용하고 싶을때 사용할 수 있게 하는 기술이라고 보면 됩니다. 설명이 더 어렵습니다.
* 클로저는 outer function 에서 호출된 함수의 변수값과 레퍼런스를 기억하고 있습니다.
* 파이썬은 함수내에 함수를 정의할 수 있다고 했습니다.

In [23]:
def outer_function(msg):
    def inner_function():
        # 여기서 msg 는 실제 inner_function 에서 선언되지 않은 outer_function 에서 넘어온 인자이지만
        # inner_function 에서 사용할 수 있게 된다. 이 msg 를 프리변수라고 하며
        # 실제 inner_function 은 outer_function 이 호출되었다 해도 실행되지 않고 선언된 상태로 관리 된다.
        return "난 내부 함수인데 {} 메세지를 받았어".format(msg)
    return inner_function

c = outer_function("헬로")
print(c())
print(dir(c)) # 내부에 __closure__
print(type(c.__closure__)) # __closure__ 는 튜플형태
print(len(c.__closure__)) # len = 1
print(dir(c.__closure__[0])) # cell_contents
print(c.__closure__[0].cell_contents)

난 내부 함수인데 헬로 메세지를 받았어
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
<class 'tuple'>
1
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'cell_contents']
헬로


## 함수 실행 시간 측정하기

In [25]:
import time
def test_func():
    start_time = time.time()
    for i in range(5):
        time.sleep(0.1)
    print("함수 동작시간: {}".format(time.time() - start_time))

    
test_func()

함수 동작시간: 0.5023365020751953


### 데코레이터 활용

In [27]:
import time
def time_check(fn_source): # fn_source 는 외부 함수 포인터(데코레이터 되는 대상 함수가 된다.)
    # 원래 함수가 받을 모든 인자를 그대로 취해야 하기 때문에 인자를 모두 수용
    # 여기서 래핑 된 함수가 클로저 함수처럼 동작
    def wrapper(*args, **kwargs): 
        start_time = time.time()
        result = fn_source(*args, **kwargs) # 실제 오리지널 함수는 여기서 실행된다.
        end_time = time.time()
        print("함수 {} 동작시간: {}".format(fn_source.__name__, end_time-start_time))
        return result # 오리지널 함수의 결과를 리턴한다.
    return wrapper # time_check 함수가 리턴될때 실제 wrapper 함수가 실행되는 개념

@time_check
def test_func():
    for i in range(5):
        time.sleep(0.1)

@time_check
def test_func2():
    for i in range(3):
        time.sleep(0.1)

test_func()
test_func2()

함수 test_func 동작시간: 0.5022459030151367
함수 test_func2 동작시간: 0.30183959007263184


In [43]:
# 데코레이터를 가장 많이 사용하는때는 로그인을 해야만 하는 사이트를 만들때
# 로그인 여부를 판단하는 로직을 처리할때 모든 구간에 로그인체크를 두지 않고
# 데코레이터를 활용하여 처리하는 경우

def login_required(function):
    def wrapper(*args, **kwargs):
        if not kwargs.get("is_login"):
            return "로그인이 필요한 페이지 입니다."
        return function(*args, **kwargs)
    return wrapper

@login_required
def test():
    print("안녕")
    
    
test()

'로그인이 필요한 페이지 입니다.'

## 함수의 속성이 사라지는 문제점
* 데코레이터로 작성을 하면 원래 함수가 갖고 있던 속성 __name__, __doc__ 등이 사라지는 문제가 생깁니다.
* 그걸 방지하기 위해선 functools 모듈의 @wraps를 데코레이터 해야 합니다.

In [51]:
def im_decorator(function):
    def wrapper(*args, **kwargs):
        return "데코레이터 {}".format(function(*args, **kwargs))
    return wrapper

@im_decorator
def test():
    '''테스트 함수'''
    return "Hello"

print(test())
print(test.__name__)
print(test.__doc__)

데코레이터 Hello
wrapper
None


In [53]:
from functools import wraps

def im_decorator(function):
    @wraps(function) # wraps 데코레이트
    def wrapper(*args, **kwargs):
        return "데코레이터 {}".format(function(*args, **kwargs))
    return wrapper

@im_decorator
def test():
    '''테스트 함수'''
    return "Hello"

print(test())
print(test.__name__)
print(test.__doc__)

데코레이터 Hello
test
테스트 함수


## 데코레이터에게 인자 넘기기

In [57]:
def add_tag(new_args):
    def decorator(func):
        def wrapper(name):
            return "<{}>{}</{}>".format(new_args, func(name), new_args)
        return wrapper
    return decorator

@add_tag("html")
def test(msg):
    return "방가워 " + msg

test("홍길동")

'<html>방가워 홍길동</html>'